## **<span style="color:purple">합성곱 신경망의 구성 요소</span>** ##
---
- 7장의 밀집층에는 뉴런마다 입력 개수만큼의 가중치가 있음
  - 가중치와 절편을 랜덤하게 초기화한 다음, 에포크를 반복하면서 경사 하강법 알고리즘을 사용하여 손실이 낮아지도록 최적의 가중치와 절편을 찾아감
  
- 합성곱은 입력 데이터 전체에 가중치를 적용하는 것이 아니라, 일부에 가중치를 곱함
- 합성곱 신경망(CNN)에서는 뉴런을 필터라고 함
- 커널은 입력에 곱하는 가중치임
- 특성 맵: 합성곱 계산을 통해 얻은 출력
- 이미지 처리 분야에서 뛰어난 성능을 발휘함
- 합성곱 층을 1개 이상 사용한 인공 신경망을 합성곱 신경망이라고 함

#### **케라스 합성곱 층**
- 패딩: 입력 배열의 주위를 가상의 원소로 채우는 것 -> 출력의 크기를 설정하기 위해, 원소들이 계산에 포함되는 수를 조정
- 세임 패딩: 입력과 특성 맵의 크기를 동일하게 만들기 위해 입력 주위에 0으로 패딩하는 것
- 밸리드 패딩: 패딩 없이 순수한 입력 배열에서만 합성곱을 하여 특성 맵을 만드는 경우, 기본값임("valid")
- 스트라이드: 이동의 크기, 기본값은 1임(한 칸씩 이동)
- 풀링: 합성곱 층에서 만든 특성 맵의 크기를 줄이지만, 개수는 유지함, 가중치가 없음, 풀링 크기와 스트라이드가 같음
   - 최대 풀링: 도장을 찍은 영역에서 가장 큰 값을 고름 -> MaxPooling2D
   - 평균 풀링: 도장을 찍은 영역에서 평균값을 계산 -> AveragePooling2D

#### **합성곱 신경망의 전체 구조**
입력 -> 합성곱 층(세임 패딩) -> 풀링층 ->밀집층 -> 예측

#### **컬러 이미지를 사용한 합성곱**
- 컬러 이미지는 RGB 채널로 구성되어 있기 때문에 3차원 배열임
- 커널 크기도 3차원이어야 함, 깊이는 입력의 깊이만큼 설정함
- 케라스는 3차원 입력을 기대함 -> 2차원 배열일 경우 깊이가 1인 3차원 배열로 변환함
- 필터의 개수가 특성 맵의 깊이를 결정함
- 필터가 늘어날 수록 이미지에 있는 구체적인 특징을 찾는다고 생각하면 됨. (ex.기본적인 특징 -> 구체적인 특징)

## **<span style="color:purple">합성곱 신경망을 사용한 이미지 분류</span>** ##
---
#### **패션 MNIST 데이터 불러오기**

#### **합성곱 신경망 만들기**

#### **모델 컴파일과 훈련**

## **<span style="color:purple">합성곱 신경망의 시각화</span>** ##
---
#### **가중치 시각화**

#### **함수형 API**
- 딥러닝에서 좀 더 복잡한 모델을 만들 때는 Sequential 클래스를 사용하기 어려움

#### **특성 맵 시각화**
- 이해하기 힘듬...이 부분은 추후 다시 공부할 것